In [ ]:
!pip install kmedoids
!pip install gdown
!pip install python-mnist
!pip install pulp
!pip install scikit-learn==0.22.2 --upgrade
!pip install zoopt

In [2]:
%cd Fair-Clustering-Codebase/

c:\Users\iason\Dev\CFC\Fair-Clustering-Codebase


In [4]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is C0DD-1089

 Directory of c:\Users\iason\Dev\CFC\Fair-Clustering-Codebase

10/01/2024  04:11 ��    <DIR>          .
10/01/2024  04:11 ��    <DIR>          ..
10/01/2024  04:11 ��    <DIR>          Consensus-Fair-Clustering
10/01/2024  04:11 ��    <DIR>          fair_clustering
10/01/2024  04:11 ��            28.880 s_DIGITS.npy
10/01/2024  04:11 ��               288 s_TOY.npy
10/01/2024  04:11 ��             8.752 U_idx_DIGITS.npy
10/01/2024  04:11 ��             9.248 U_idx_MNIST_USPS.npy
10/01/2024  04:11 ��             1.160 U_idx_Office-31.npy
10/01/2024  04:11 ��               176 U_idx_TOY.npy
10/01/2024  04:11 ��             5.920 U_idx_Yale.npy
10/01/2024  04:11 ��            20.256 V_idx_DIGITS.npy
10/01/2024  04:11 ��            21.408 V_idx_MNIST_USPS.npy
10/01/2024  04:11 ��             9.440 V_idx_Office-31.npy
10/01/2024  04:11 ��               240 V_idx_TOY.npy
10/01/2024  04:11 ��            13.648 V_idx_Yale.npy
10/

In [6]:
import sys
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd
import random
import kmedoids
from sklearn.decomposition import PCA
from sklearn.random_projection import SparseRandomProjection
from zoopt import Dimension, ValueType, Objective, Parameter, Opt, ExpOpt
# import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

from fair_clustering.eval.functions import * #[TO-DO] Write base class and derive metrics from it, temporary eval code

from fair_clustering.dataset import ExtendedYaleB, Office31, MNISTUSPS
from fair_clustering.algorithm import FairSpectral, FairKCenter, FairletDecomposition, ScalableFairletDecomposition

import matplotlib.pyplot as plt

In [7]:
# Set parameters related to dataset and get dataset

name = 'Office-31' #Choose between Office-31, MNIST_USPS, Yale, or DIGITS

if name == 'Office-31':
  dataset = Office31(exclude_domain='amazon', use_feature=True)
  X, y, s = dataset.data
elif name == 'MNIST_USPS':
  dataset = MNISTUSPS(download=True)
  X, y, s = dataset.data
elif name == 'Yale':
  dataset = ExtendedYaleB(download=True, resize=True)
  X, y, s = dataset.data
elif name == 'DIGITS':
  X, y, s = np.load('X_' + name + '.npy'), np.load('y_' + name + '.npy'), np.load('s_' + name + '.npy')

print(X.shape, y.shape, s.shape)

cl_algo = 'SFD' #Choose between FSC or SFD. For KFC run notebook locally (KFC is not available in Colab due to CPLEX being incompatible with Colab)


Download domain_adaptation_features_20110616.tar.gz from https://drive.google.com/u/0/uc?id=0B4IapRTv9pJ1WTVSd2FIcW4wRTA&export=download


Downloading...
From: https://drive.google.com/u/0/uc?id=0B4IapRTv9pJ1WTVSd2FIcW4wRTA&export=download
To: c:\Users\iason\.conda\envs\fact2024\python37.zip\fair_clustering\raw_data\office31\domain_adaptation_features_20110616.tar.gz
100%|██████████| 1.44M/1.44M [00:00<00:00, 29.3MB/s]


Download office31_resnet50.zip from https://wjdcloud.blob.core.windows.net/dataset/office31_resnet50.zip


Downloading...
From: https://wjdcloud.blob.core.windows.net/dataset/office31_resnet50.zip
To: c:\Users\iason\.conda\envs\fact2024\python37.zip\fair_clustering\raw_data\office31\office31_resnet50.zip
100%|██████████| 227M/227M [00:53<00:00, 4.24MB/s] 


(1293, 2048) (1293,) (1293,)


In [ ]:
# Fairness Attack

In [1]:
def attack_balance(solution):
  X_copy, s_copy = X.copy(), s.copy()
  flipped_labels = solution.get_x()
  i = 0
  for idx in U_idx:
    s_copy[idx] = flipped_labels[i]
    i += 1

  if cl_algo == 'FSC':
    if name == 'MNIST_USPS':
      metr_str = 'manhattan'
    else:
      metr_str = 'euclidean'
    fair_clustering_algo = FairSpectral(n_clusters=n_clusters, num_neighbors=3, metric_str=metr_str, random_state=random_state)
  if cl_algo =='SFD':
    if name == 'DIGITS':      
      fair_clustering_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=1, random_state=random_state) 
    else:
      fair_clustering_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=2, random_state=random_state) 
  
  fair_clustering_algo.fit(X_copy, s_copy)
  labels_sfd = fair_clustering_algo.labels_

  s_eval = []
  X_eval = []
  labels_sfd_eval = []
  for idx in V_idx:
    s_eval.append(s_copy[idx])
    X_eval.append(X_copy[idx])
    labels_sfd_eval.append(labels_sfd[idx])
  s_eval = np.array(s_eval)
  X_eval = np.array(X_eval)
  labels_sfd_eval = np.array(labels_sfd_eval)

  bal = balance(labels_sfd_eval, X_eval, s_eval)

  return bal


def attack_entropy(solution):
  X_copy, s_copy = X.copy(), s.copy()
  flipped_labels = solution.get_x()
  i = 0
  for idx in U_idx:
    s_copy[idx] = flipped_labels[i]
    i += 1

  if cl_algo == 'FSC':
    if name == 'MNIST_USPS':
      metr_str = 'manhattan'
    else:
      metr_str = 'euclidean'
    fair_clustering_algo = FairSpectral(n_clusters=n_clusters, num_neighbors=3, metric_str=metr_str, random_state=random_state)
  if cl_algo =='SFD':
    if name == 'DIGITS':      
      fair_clustering_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=1, random_state=random_state) 
    else:
      fair_clustering_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=2, random_state=random_state) 
  
  fair_clustering_algo.fit(X_copy, s_copy)
  labels_sfd = fair_clustering_algo.labels_

  s_eval = []
  X_eval = []
  labels_sfd_eval = []
  for idx in V_idx:
    s_eval.append(s_copy[idx])
    X_eval.append(X_copy[idx])
    labels_sfd_eval.append(labels_sfd[idx])
  s_eval = np.array(s_eval)
  X_eval = np.array(X_eval)
  labels_sfd_eval = np.array(labels_sfd_eval)

  ent = entropy(labels_sfd_eval, s_eval)

  return ent

In [2]:
def process_solution(sol):
  X_copy, s_copy, y_copy = X.copy(), s.copy(), y.copy()
  flipped_labels = sol.get_x()
  i = 0
  for idx in U_idx:
    s_copy[idx] = flipped_labels[i]
    i += 1

  if cl_algo == 'FSC':
    if name == 'MNIST_USPS':
      metr_str = 'manhattan'
    else:
      metr_str = 'euclidean'
    fair_clustering_algo = FairSpectral(n_clusters=n_clusters, num_neighbors=3, metric_str=metr_str, random_state=random_state)
  if cl_algo =='SFD':
    if name == 'DIGITS':      
      fair_clustering_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=1, random_state=random_state)
    else:
      fair_clustering_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=2, random_state=random_state) 
  
  fair_clustering_algo.fit(X_copy, s_copy)
  labels_sfd = fair_clustering_algo.labels_

  s_eval = []
  X_eval = []
  labels_sfd_eval = []
  y_eval = []
  for idx in V_idx:
    s_eval.append(s_copy[idx])
    X_eval.append(X_copy[idx])
    labels_sfd_eval.append(labels_sfd[idx])
    y_eval.append(y_copy[idx])
  s_eval = np.array(s_eval)
  X_eval = np.array(X_eval)
  labels_sfd_eval = np.array(labels_sfd_eval)
  y_eval = np.array(y_eval)

  bal = balance(labels_sfd_eval, X_eval, s_eval)
  ent = entropy(labels_sfd_eval, s_eval)
  accuracy = acc(y_eval, labels_sfd_eval)
  nmi_score = nmi(y_eval, labels_sfd_eval)

  return (bal, ent, accuracy, nmi_score)

In [3]:
def conduct_random_attack(size_sol):
  X_copy, s_copy, y_copy = X.copy(), s.copy(), y.copy()
  random.seed(None)
  flipped_labels = [random.randint(0,1) for _ in range(size_sol)]
  i = 0
  for idx in U_idx:
    s_copy[idx] = flipped_labels[i]
    i += 1

  if cl_algo == 'FSC':
    if name == 'MNIST_USPS':
      metr_str = 'manhattan'
    else:
      metr_str = 'euclidean'
    fair_clustering_algo = FairSpectral(n_clusters=n_clusters, num_neighbors=3, metric_str=metr_str, random_state=random_state)
  if cl_algo =='SFD':
    if name == 'DIGITS':      
      fair_clustering_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=1, random_state=random_state) #5,2
    else:
      fair_clustering_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=2, random_state=random_state) #5,2

  fair_clustering_algo.fit(X_copy, s_copy)
  labels_sfd = fair_clustering_algo.labels_

  s_eval = []
  X_eval = []
  labels_sfd_eval = []
  y_eval = []
  for idx in V_idx:
    s_eval.append(s_copy[idx])
    X_eval.append(X_copy[idx])
    labels_sfd_eval.append(labels_sfd[idx])
    y_eval.append(y_copy[idx])
  s_eval = np.array(s_eval)
  X_eval = np.array(X_eval)
  labels_sfd_eval = np.array(labels_sfd_eval)
  y_eval = np.array(y_eval)

  bal = balance(labels_sfd_eval, X_eval, s_eval)
  ent = entropy(labels_sfd_eval, s_eval)
  accuracy = acc(y_eval, labels_sfd_eval)
  nmi_score = nmi(y_eval, labels_sfd_eval)

  return (bal, ent, accuracy, nmi_score)

In [4]:
import numpy as np
n_clusters = len(np.unique(y))
print("# of clusters -> " + str(n_clusters))
seeds = [150, 1, 4200, 424242, 1947, 355, 256, 7500, 99999, 18]
n_trials = len(seeds)

U_idx_full, V_idx_full = np.load('U_idx_' + name + '.npy').tolist(), np.load('V_idx_' + name + '.npy').tolist()

pre_attack_res = {
    0 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    1 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    2 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    3 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    4 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    5 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    6 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    7 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
}

post_attack_res = {
    0 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    1 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    2 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    3 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    4 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    5 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    6 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    7 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
}

random_attack_res = {
    0 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    1 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    2 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    3 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    4 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    5 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    6 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
    7 : {'BALANCE': [], 'ENTROPY': [], 'ACC': [], 'NMI': []},
}

for percent, j in enumerate([int(0.125*len(U_idx_full)), int(0.25*len(U_idx_full)), int(0.375*len(U_idx_full)), int(0.5*len(U_idx_full)), int(0.625*len(U_idx_full)), int(0.75*len(U_idx_full)), int(0.875*len(U_idx_full)), int(len(U_idx_full))]):
  
  U_idx = U_idx_full[:j]
  V_idx = V_idx_full

  for trial_idx in range(n_trials):
    random_state = seeds[trial_idx]
    
    if cl_algo == 'FSC':
      if name == 'MNIST_USPS':
        metric_string = 'manhattan'
      else:
        metric_string = 'euclidean'
      fair_algo = FairSpectral(n_clusters=n_clusters, num_neighbors=3, metric_str = metric_string, random_state=random_state)
      fair_algo.fit(X, s)
      labels = fair_algo.labels_
    if cl_algo =='SFD':
      if name == 'DIGITS':      
        fair_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=1, random_state=random_state) #5,2
      else:
        fair_algo = ScalableFairletDecomposition(n_clusters=n_clusters, alpha=5, beta=2, random_state=random_state) #5,2
      fair_algo.fit(X, s)
      labels = fair_algo.labels_
    

    s_test = []
    X_test = []
    labels_test = []
    y_test = []
    for idx in V_idx:
      s_test.append(s[idx])
      X_test.append(X[idx])
      labels_test.append(labels[idx])
      y_test.append(y[idx])
    s_test = np.array(s_test)
    X_test = np.array(X_test)
    labels_test = np.array(labels_test)
    y_test = np.array(y_test)

    
    pre_attack_res[percent]['BALANCE'].append(balance(labels_test, X_test, s_test))
    pre_attack_res[percent]['ENTROPY'].append(entropy(labels_test, s_test))
    pre_attack_res[percent]['ACC'].append(acc(y_test, labels_test))
    pre_attack_res[percent]['NMI'].append(nmi(y_test, labels_test))
    
    dim_size = len(U_idx)
    dim = Dimension(dim_size, [[0, 1]]*dim_size, [False]*dim_size)

    if name == 'Office-31': #Only attack_balance
      obj = Objective(attack_balance, dim)
    elif name == 'MNIST_USPS' or name == 'DIGITS':
      if cl_algo == 'SFD':
        obj = Objective(attack_balance, dim)
      elif cl_algo == 'FSC':
        obj = Objective(attack_entropy, dim)
    elif name == 'Yale': #Only attack_entropy
      obj = Objective(attack_entropy, dim)
    
    solution = Opt.min(obj, Parameter(budget=10)) # 10 for FSC for MNIST_USPS and 50 for SFD for MNIST_USPS || 20 for FSC for Office-31 and 20 for SFD for Office-31 || 10 for FSC for Yale and 20 for SFD for Yale || 15 for FSC for DIGITS and 25 for SFD for DIGITS

    
    pa_bal, pa_ent, pa_acc, pa_nmi = process_solution(solution)
    post_attack_res[percent]['BALANCE'].append(pa_bal)
    post_attack_res[percent]['ENTROPY'].append(pa_ent)
    post_attack_res[percent]['ACC'].append(pa_acc)
    post_attack_res[percent]['NMI'].append(pa_nmi)

    r_bal, r_ent, r_acc, r_nmi = conduct_random_attack(dim_size)
    random_attack_res[percent]['BALANCE'].append(r_bal)
    random_attack_res[percent]['ENTROPY'].append(r_ent)
    random_attack_res[percent]['ACC'].append(r_acc)
    random_attack_res[percent]['NMI'].append(r_nmi)


NameError: name 'np' is not defined